# <span style = 'color: red'>**Práctica tres.** Aprendizaje de máquina. *Clusterización*</span>

# ***Parte dos.*** **Comparación de métodos de clusterización.**

## **Simón Cuartas Rendón**

En este *notebook* de Python se aborda el desarrollo de la **segunda parte** de la *práctica tres* del curso de **Aprendizaje de Máquina**, relacionada con la clusterización vía *KMeans*, clusterización aglomerativa, DBSCAN y clusterización espectral. Para cada una de estas alternativas se va a jugar con algunos de sus parámetros de cara a ver cuál consigue optimizar el índice de silueta.

A continuación, se abren los paquetes que se usarán para dar solución al problema propuesto.

**Anotación**. Este *notebook* está disponible en línea para facilitar su revisión. Se puede acceder al repositorio de *GitHub* aquí, y en particular, a [este](https://github.com/scuartasr/aprendmaq_entregable_3) cuaderno aquí XXXX.